In [5]:
import os
import numpy as np

import cv2

In [6]:
image_paths = os.listdir("dataset/MSFD/1/face_crop")
ground_truths = os.listdir("dataset/MSFD/1/face_crop_segmentation")

In [7]:
if os.path.exists("Output/PartC"):

    for file in os.listdir("Output/PartC"): 
        file_path = os.path.join("Output/PartC", file)

        if os.path.isfile(file_path):
            os.unlink(file_path)

else:
    os.makedirs("Output/PartC")

In [8]:
def evaluate(segment, ground_truth):
    
    intersection = ((segment > 0) & (ground_truth > 0)).sum()
    union = ((segment > 0) | (ground_truth > 0)).sum() + 1e-8        
    iou = (intersection / union).mean()

    return iou

In [9]:
image_num = 1
sum_iou = 0

for index in range(min(500, len(image_paths))):

    image_path = image_paths[index]
    ground_truth_path = ground_truths[index]

    print(f"Processing Image {image_num}...")

    image = cv2.imread("dataset/MSFD/1/face_crop/" + image_path)
    ground_truth = cv2.imread("dataset/MSFD/1/face_crop_segmentation/" + image_path)

    if image is None or ground_truth is None:
        continue

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blurred_image = cv2.GaussianBlur(gray_image, (3, 3), 0.5)
    thresh, binary_image = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    edges = cv2.Canny(blurred_image, threshold1=20, threshold2=100)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    image_width, image_height = image.shape[:2]
    image_size = image_width * image_height
    
    segmented_image1 = np.zeros([image_width, image_height, 3], dtype=float)
    segmented_image2 = np.zeros([image_width, image_height, 3], dtype=float)
    
    for cnt in contours:

        x, y, w, h = cv2.boundingRect(cnt)
        
        _, inv_binary_segment = cv2.threshold(blurred_image[y: y + h, x: x + w], 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        _, binary_segment = cv2.threshold(blurred_image[y: y + h, x: x + w], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        for i in range(y, y + h):
            for j in range(x, x + w):
                segmented_image1[i][j] += binary_segment[i - y][j - x]
                segmented_image2[i][j] += inv_binary_segment[i - y][j - x]

    cut = int(0.6 * image_height)

    segmented_image1[: cut, :, :] = 0
    segmented_image2[: cut, :, :] = 0
    
    sum_lower1 = segmented_image1.sum()
    sum_lower2 = segmented_image2.sum()

    if sum_lower1 > sum_lower2:
        cv2.imwrite(f"Output/PartC/{image_path}.png", segmented_image1)
        sum_iou += evaluate(segmented_image1, ground_truth)

    else:
        cv2.imwrite(f"Output/PartC/{image_path}.png", segmented_image2)
        sum_iou += evaluate(segmented_image2, ground_truth)

    print(f"Image {image_num} processed.")
    image_num += 1

print(sum_iou / (image_num - 1))


Processing Image 1...
Image 1 processed.
Processing Image 2...
Image 2 processed.
Processing Image 3...
Image 3 processed.
Processing Image 4...
Image 4 processed.
Processing Image 5...
Image 5 processed.
Processing Image 6...
Image 6 processed.
Processing Image 7...
Image 7 processed.
Processing Image 8...
Image 8 processed.
Processing Image 9...
Image 9 processed.
Processing Image 10...
Image 10 processed.
Processing Image 11...
Image 11 processed.
Processing Image 12...
Image 12 processed.
Processing Image 13...
Image 13 processed.
Processing Image 14...
Image 14 processed.
Processing Image 15...
Image 15 processed.
Processing Image 16...
Image 16 processed.
Processing Image 17...
Image 17 processed.
Processing Image 18...
Image 18 processed.
Processing Image 19...
Image 19 processed.
Processing Image 20...
Image 20 processed.
Processing Image 21...
Image 21 processed.
Processing Image 22...
Image 22 processed.
Processing Image 23...
Image 23 processed.
Processing Image 24...
Image 